## Flask JavaScript Ajax Examples
----
* Adding in the Other JavasScript Examples to Build out the Flask Examples JS library
* starting off with `setup.py`:

```python
import io

from setuptools import find_packages
from setuptools import setup

with io.open("README.rst", "rt", encoding="utf8") as f:
    readme = f.read()

setup(
    name="js_example",
    version="1.0.0",
    url="http://flask.pocoo.org/docs/patterns/jquery/",
    license="BSD",
    maintainer="Pallets team",
    maintainer_email="contact@palletsprojects.com",
    description="Demonstrates making Ajax requests to Flask.",
    long_description=readme,
    packages=find_packages(),
    include_package_data=True,
    zip_safe=False,
    install_requires=["flask"],
    extras_require={"test": ["pytest", "coverage", "blinker"]},
)
```

### Next create a `templates` folder and add `__init__.py`
----
* `mkdir templates` to hold the templates
* add a `__init__.py` to initialize the application

```python
from flask import Flask

app = Flask(__name__)

from js_example import views
```
 

* add `views.py`

```python

Learn more or give us feedback
from flask import jsonify
from flask import render_template
from flask import request

from js_example import app


@app.route("/", defaults={"js": "plain"})
@app.route("/<any(plain, jquery, fetch):js>")
def index(js):
    return render_template("{0}.html".format(js), js=js)


@app.route("/add", methods=["POST"])
def add():
    a = request.form.get("a", 0, type=float)
    b = request.form.get("b", 0, type=float)
    return jsonify(result=a + b

```

### `base.html`

```html
<!doctype html>
<title>JavaScript Example</title>
<link rel="stylesheet" href="https://unpkg.com/sakura.css@1.0.0/css/normalize.css">
<link rel="stylesheet" href="https://unpkg.com/sakura.css@1.0.0/css/sakura-earthly.css">
<style>
  ul { margin: 0; padding: 0; display: flex; list-style-type: none; }
  li > * { padding: 1em; }
  li.active > a { color: #5e5e5e; border-bottom: 2px solid #4a4a4a; }
  form { display: flex; }
  label > input { width: 3em; }
  form > * { padding-right: 1em; }
  #result { font-weight: bold; }
</style>
<ul>
  <li><span>Type:</span>
  <li class="{% if js == 'plain' %}active{% endif %}">
    <a href="{{ url_for('index', js='plain') }}">Plain</a>
  <li class="{% if js == 'fetch' %}active{% endif %}">
    <a href="{{ url_for('index', js='fetch') }}">Fetch</a>
  <li class="{% if js == 'jquery' %}active{% endif %}">
    <a href="{{ url_for('index', js='jquery') }}">jQuery</a>
</ul>
<hr>
<p>{% block intro %}{% endblock %}</p>
<hr>
<form id="calc">
  <label>a <input name="a"></label>
  <span>+</span>
  <label>b <input name="b"></label>
  <input type="submit" value="Calculate">
</form>
<span>= <span id="result"></span></span>
{% block script %}{% endblock %}

```

### `fetch.html`
----
```html
{% extends 'base.html' %}

{% block intro %}
  <a href="https://developer.mozilla.org/en-US/docs/Web/API/WindowOrWorkerGlobalScope/fetch"><code>fetch</code></a>
  is the <em>new</em> plain JavaScript way to make requests. It's
  supported in all modern browsers except IE, which requires a
  <a href="https://github.com/github/fetch">polyfill</a>.
{% endblock %}

{% block script %}
  <script src="https://unpkg.com/promise-polyfill@7.1.2/dist/polyfill.min.js"></script>
  <script src="https://unpkg.com/whatwg-fetch@2.0.4/fetch.js"></script>
  <script>
    function addSubmit(ev) {
      ev.preventDefault();
      fetch({{ url_for('add')|tojson }}, {
        method: 'POST',
        body: new FormData(this)
      })
        .then(parseJSON)
        .then(addShow);
    }
    function parseJSON(response) {
      return response.json();
    }
    function addShow(data) {
      var span = document.getElementById('result');
      span.innerText = data.result;
    }
    var form = document.getElementById('calc');
    form.addEventListener('submit', addSubmit);
  </script>
{% endblock %}
```

### `jquery.html`
-----
```html
{% extends 'base.html' %}

{% block intro %}
  <a href="https://jquery.com/">jQuery</a> is a popular library that
  adds cross browser APIs for common tasks. However, it requires loading
  an extra library.
{% endblock %}

{% block script %}
  <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
  <script>
    function addSubmit(ev) {
      ev.preventDefault();
      $.ajax({
        method: 'POST',
        url: {{ url_for('add')|tojson }},
        data: $(this).serialize()
      }).done(addShow);
    }
    function addShow(data) {
      $('#result').text(data.result);
    }
    $('#calc').on('submit', addSubmit);
  </script>
{% endblock %}
```


### `plain.html`
----

```html
{% extends 'base.html' %}

{% block intro %}
  <a href="https://developer.mozilla.org/en-US/docs/Web/API/XMLHttpRequest"><code>XMLHttpRequest</code></a>
  is the plain JavaScript way to make requests. It's natively supported
  by all browsers.
{% endblock %}

{% block script %}
  <script>
    function addSubmit(ev) {
      ev.preventDefault();
      var request = new XMLHttpRequest();
      request.addEventListener('load', addShow);
      request.open('POST', {{ url_for('add')|tojson }});
      request.send(new FormData(this));
    }
    function addShow() {
      var data = JSON.parse(this.responseText);
      var span = document.getElementById('result');
      span.innerText = data.result;
    }
    var form = document.getElementById('calc');
    form.addEventListener('submit', addSubmit);
  </script>
{% endblock %}
```